In [2]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
#tf.set_random_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

In [3]:
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

In [4]:
rcParams['figure.figsize'] = 12, 5

In [9]:
path = '1661-0.txt'
text = open(path,encoding="utf8").read().lower()
print('corpus length:', len(text))

corpus length: 581886


In [10]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [11]:
SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(len(sentences))

193949


In [12]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [13]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [14]:
print("gere")

gere


In [15]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20
1440/1440 [==============================] - 87s 59ms/step - loss: 2.2555 - accuracy: 0.3516 - val_loss: 2.1174 - val_accuracy: 0.4314
Epoch 2/20
1440/1440 [==============================] - 88s 61ms/step - loss: 1.6361 - accuracy: 0.5107 - val_loss: 2.0576 - val_accuracy: 0.4342
Epoch 3/20
1440/1440 [==============================] - 88s 61ms/step - loss: 1.5116 - accuracy: 0.5442 - val_loss: 2.0449 - val_accuracy: 0.4470
Epoch 4/20
1440/1440 [==============================] - 89s 62ms/step - loss: 1.4461 - accuracy: 0.5612 - val_loss: 2.0330 - val_accuracy: 0.4542
Epoch 5/20
1440/1440 [==============================] - 88s 61ms/step - loss: 1.4146 - accuracy: 0.5711 - val_loss: 2.0413 - val_accuracy: 0.4535
Epoch 6/20
1440/1440 [==============================] - 86s 60ms/step - loss: 1.3950 - accuracy: 0.5755 - val_loss: 2.0457 - val_accuracy: 0.4595
Epoch 7/20
1440/1440 [==============================] - 87s 61ms/step - loss: 1.3629 - accuracy: 0.5848 - val_loss: 2.0607 -

In [16]:
model.save('keras_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [17]:
model = load_model('keras_model.h5')
history = pickle.load(open("history.p", "rb"))

In [18]:
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
        
    return x

In [19]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [20]:
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion

In [21]:
def predict_completions(text, n=3):
    x = prepare_input(text)
    print("gf")
    preds = model.predict(x, verbose=0)[0]
    print("fg")
    next_indices = sample(preds, n)
    print(text)
    print(predict_completion(text[1:]))
#     print(indices_char[16]+predict_completion(text[1:]+indices_char[16]))
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]

In [22]:
quotes = [
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]

In [23]:
for q in quotes:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 5))
    print()

it is not a lack of love, but a lack of 
gf
fg
it is not a lack of love, but a lack of 
bsommsfom 
['the ', 'a ', 'some ', 'his ', 'conserks ']

that which does not kill us makes us str
gf
fg
that which does not kill us makes us str
ag 
['ong ', 'ike ', 'eet, ', 'uck ', 'ange ']

i'm not upset that you lied to me, i'm u
gf
fg
i'm not upset that you lied to me, i'm u
bmlso 
['nder ', 'pon ', 'sed ', 'lle ', 'ttical ']

and those who were seen dancing were tho
gf
fg
and those who were seen dancing were tho
emsab-ifbdaui 
['ught ', 'ment ', 'se ', 'ok ', 'ing ']

it is hard enough to remember my opinion
gf
fg
it is hard enough to remember my opinion
ussl 
['. ', ', ', ' of ', '\nwhich ', '?’\n\n“‘i ']

